# Spiral Galaxies

*Colin Leach, Nov 2018*

## Contents

- [Rotation curves](#rotcurves)
- [Out-of-plane and radial motion](#othermotions)
    - [Visualizing epicycles](#visepi)
- [Making spirals](#spirals)
    - [The winding problem](#winding)
    - [Ovals within ovals](#ovals)

<a id='rotcurve'></a>

## Rotation curves

***TODO***

In [13]:
%matplotlib inline

import time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# from IPython.display import Image, HTML, display, clear_output
# from matplotlib.offsetbox import AnchoredText

from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Output
import ipywidgets as w

# make graph labels more readable
plt.rcParams.update({'font.size': 16})

<a id='othermotions'></a>

## Out-of-plane and radial motion

Though stars follow an approximately circular, in-plane path around the galaxy following the minimum in the effective gravitational potential $\Phi_{eff}$, there are a variety of small-amplitude deviations. These are complicated in detail but can be approximated as simple harmonic motion.

For vertical motions, the oscillation follows $\ddot{z} \approx -\nu^2 z$, with a minimum at the midplane and frequency $\nu$.

For radial motions, the minimum is at $R_0$ where $\frac{d\Phi_{eff}}{dR}=0$. Defining $\rho(t) \equiv (R(t)-R_0)$, the oscillation is $\ddot{\rho} = -\kappa^2 \rho$. Here $\kappa$ is the frequency of oscillation, but because of some mathematical similarities to planetary motions in the Ptolemeic system it is often know as *epicycle frequency*.

The frequencies are defined by the 2nd derivative of $\Phi_{eff}$, evaluated at the minimum:

$$ \nu^2 = \frac{\partial^2\Phi_{eff}}{\partial z^2} \bigg|_{midplane} \qquad \kappa^2 = \frac{\partial^2\Phi_{eff}}{\partial R^2} \bigg|_{R_0} $$

In both cases, the frequencies can be arbitrary and will in general give open orbits. Having an oscillation period which is in a simple integer relationship to the galactic orbit period is a special case: now the star will return to its original position, in a closed orbit.

<a id='visepi'></a>

### Visualizing epicycles

What would radial oscillations look like if viewed from above the galactic plane? Probably too small to notice, but we can exaggerate that.

In [28]:
def plot_epicycles(ratio, amplitude):
    "ratio: number of radial oscillations per galactic orbit"
    
    nPoints = 100
    thetas = np.linspace(0, 4*np.pi, 200)
    rs = 1 + amplitude * np.sin(ratio*thetas)
    xs = rs*np.cos(thetas)
    ys = rs*np.sin(thetas)
    
    plt.figure(figsize=(9, 9))
    plt.plot(xs, ys)
    plt.title('Epicycles with {} oscillations per orbit, amplitude {} $R_0$'.format(ratio, amplitude))   

style = {'description_width': 'initial'} # to avoid the labels getting truncated
interact(plot_epicycles, 
             ratio = w.FloatSlider(description="Oscillations per orbit", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False,
                                            min=1.0, max=8.0, 
                                            value=3),
             amplitude = w.FloatSlider(description="Amplitude, units of $R_0$", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False, 
                                            min=0.01, max=0.3, step=0.01,
                                            value=0.1));

interactive(children=(FloatSlider(value=3.0, continuous_update=False, description='Oscillations per orbit', la…

<a id='spirals'></a>

## Making spirals

The spirals are visually spectacular and the location of a lot of star formation, but how do they form and how do they persist for billions of years?

We know that for stars within the disk, $\dot{\phi(R)} \ne constant$, so what is the effect of differential rotation?

<a id='winding'></a>

### The winding problem

Imagine we start with a straight line of stars across the galactic diameter, and let it evolve through time as the stars at their individual velocities. 

To implement this, we need an efficient way to rotate points through arbitrary angles in Cartesian coordinates. Some simple linear algebra will do the trick.

The rotation matrix rotates a point in 2-D space anticlockwise about the origin by angle $\theta$:
$$ \begin{pmatrix} \cos \theta & -\sin \theta \\ \sin \theta & \cos \theta \end{pmatrix} 
\begin{pmatrix} x_0 \\ y_0 \end{pmatrix} = \begin{pmatrix} x_1 \\ y_1 \end{pmatrix} $$
Note that $| \mathbf{x_0} | = | \mathbf{x_1} |$, so the radius is unchanged.

In [6]:
def rotation(theta):
    # theta: angle in degrees
    # returns: 2x2 rotation matix as numpy array
    theta_rad = theta*np.pi/180
    return np.array([[np.cos(theta_rad), -np.sin(theta_rad)],[np.sin(theta_rad), np.cos(theta_rad)]])

This can multiply $2 \times N$ arrays directly, providing the angle is the same throughout (see the Ovals demo below). However, the winding problem is all about differential rotation so the code gets a bit clumsier.

As a simple model, assume that we ignore the core of the galaxy (the inner 10% of the radius), and outside that the linear velocity $v$ is constant. Then the angular velocity $\omega(r) = v/r$. The plot routine takes an angle (in degrees) for rotation of the outer edge.

Reminder for the old Python 2 programmers: since version 3.5 Python has the @ operator for matrix multiplication. It is no longer necessary to call `numpy.matmul()` explicitly.

In [7]:
def plotWinding(phi_step):
    nPoints = 1000
    startx = np.linspace(0.1, 1, nPoints)
    starty = np.zeros(nPoints)
    startline = np.stack([startx, starty])

    phis = phi_step*np.ones(nPoints)/startx
    rots = rotation(phis)
    newline = np.zeros((2,nPoints))
    for i in range(nPoints):
        newline[:,i] = rots[:,:,i] @ startline[:,i]

    fig = plt.figure(figsize=(8,8))
    plt.axis('equal')
    plt.plot(newline[0,:], newline[1,:], 'b-')
    plt.plot(-newline[0,:], -newline[1,:], 'r-')

In [8]:
style = {'description_width': 'initial'} # to avoid the labels getting truncated
interact(plotWinding, 
             phi_step = w.IntSlider(description="Degrees rotation", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False, 
                                            min=0, max=150,
                                            value=0) );

interactive(children=(IntSlider(value=0, continuous_update=False, description='Degrees rotation', layout=Layou…

Clearly, in this model the spirals rapidly wind up tighter and tighter, on a timescale much shorter than the lifetime of a spiral galaxy. This doesn't match observations, so we need a different model.

<a id='ovals'></a>

### Ovals within ovals

We should probably think of the spiral arms as patterns rather than structures. 

In [9]:
def cartesianCircle(r = 1):
    phis = np.linspace(0, 2*np.pi, 100)
    x = r*np.cos(phis)
    y = r*np.sin(phis)
    return x, y

def squashCircle(circ, b):
    # scale down y-axis by factor b, leaving x-axis unchanged
    M = np.array([[1,0],[0,b]])
    return M @ circ

def scale(c):
    # scale by axes by a factor c
    return np.array([[c,0],[0,c]])

Define a plot of `nOvals` ellipses, each a factor `dr` larger than the one inside and rotated `dtheta` degrees anticlockwise. 

In [10]:
def plotOvals(nOvals=25, dtheta=5, dr=0.05, axratio=0.7):
    circ = cartesianCircle()
    ell = squashCircle(circ, axratio)
    fig = plt.figure(figsize=(8,8))
    plt.axis('equal')
    plt.plot(ell[0,:], ell[1,:], '-', color='tab:gray')
    for n in range(1,nOvals):
        newell = rotation(dtheta*n) @ scale(1+dr*n) @ ell
        plt.plot(newell[0,:], newell[1,:], '-', color='tab:gray')


In [11]:
style = {'description_width': 'initial'} # to avoid the labels getting truncated
interact(plotOvals, 
             nOvals = w.IntSlider(description="Ovals to plot", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False,
                                            min=5, max=100, 
                                            value=25), 
             dtheta = w.FloatSlider(description="Rotation step (deg)", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False,
                                            min=0.1, max=20, 
                                            value=5.0),
             dr = w.FloatSlider(description="Scale step", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False, 
                                            min=0, max=0.5, step=0.01,
                                            value=0.05),
             axratio = w.FloatSlider(description="Axis ratio", style=style,
                                            layout=Layout(width='80%'),
                                            continuous_update=False, 
                                            min=0.5, max=0.9, step=0.01,
                                            value=0.8)
        );


interactive(children=(IntSlider(value=25, continuous_update=False, description='Ovals to plot', layout=Layout(…

So we can make quite conspicuous spiral arms appear without explicitly drawing them. 